In [1]:
!pip install -r requirements.txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 817.7/817.7 kB 16.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 171.5/171.5 kB 14.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.5/85.5 MB 11.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.0/27.0 MB 36.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.6/297.6 kB 33.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.1/199.1 kB 24.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 MB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 245.2/245.2 kB 29.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 84.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 65.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 299.3/299.3 kB 39.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━

In [ ]:
import PyPDF2

from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.llms import LlamaCpp


from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationalRetrievalChain


from langchain.prompts import PromptTemplate
from sentence_transformers import SentenceTransformer, util
from langchain.callbacks.manager import CallbackManager
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler

import transformers

from huggingface_hub import login
from google.colab import userdata

token = userdata.get('HF_TOKEN')
login(token=token)

from transformers import BitsAndBytesConfig, AutoTokenizer,AutoModelForCausalLM

from langchain.text_splitter import CharacterTextSplitter
from langchain.document_transformers import Html2TextTransformer
from langchain.document_loaders import AsyncChromiumLoader

from langchain.embeddings.huggingface import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS

from langchain.prompts import PromptTemplate
from langchain.schema.runnable import RunnablePassthrough
from langchain.llms import HuggingFacePipeline
from langchain.chains import LLMChain

import torch

import os
from transformers import (
  AutoTokenizer,
  AutoModelForCausalLM,
  BitsAndBytesConfig,
  pipeline
)

from transformers import BitsAndBytesConfig

from langchain.text_splitter import CharacterTextSplitter
from langchain.document_transformers import Html2TextTransformer
from langchain.document_loaders import AsyncChromiumLoader

from langchain.embeddings.huggingface import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS

from langchain.prompts import PromptTemplate
from langchain.schema.runnable import RunnablePassthrough
from langchain.llms import HuggingFacePipeline
from langchain.chains import LLMChain

from langchain.text_splitter import CharacterTextSplitter
from langchain.document_transformers import Html2TextTransformer
from langchain.document_loaders import AsyncChromiumLoader

from langchain.embeddings.huggingface import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS

from langchain.prompts import PromptTemplate
from langchain.schema.runnable import RunnablePassthrough
from langchain.llms import HuggingFacePipeline

from langchain.schema import format_document
from langchain_core.messages import get_buffer_string
from langchain_core.runnables import RunnableLambda, RunnablePassthrough
from langchain.memory import ConversationBufferMemory
from langchain_core.prompts.chat import ChatPromptTemplate

from operator import itemgetter

Token has not been saved to git credential helper. Pass `add_to_git_credential=True` if you want to set the git credential as well.
Token is valid (permission: read).
Your token has been saved to /root/.cache/huggingface/token
Login successful


##### Step 1: Load the Retriever and Vector Database from Part A

In [4]:
import os
from google.colab import drive
drive.mount('/content/drive')

def load_vectordb_from_local():
    embeddings = HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2', model_kwargs={'device': 'cpu'})
    DB_FAISS_PATH = '/content/drive/My Drive/vector_db'
    if os.path.exists(DB_FAISS_PATH):
        db = FAISS.load_local('/content/drive/My Drive/vector_db',embeddings, allow_dangerous_deserialization=True)
        return db
    else:
        print("VectorDB not found at the specified path.")
        return None

loaded_db = load_vectordb_from_local()
if loaded_db:
    print("VectorDB loaded successfully!")
else:
    print("Failed to load VectorDB.")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
VectorDB loaded successfully!


In [5]:
retriever = loaded_db.as_retriever()

##### Step 2: Load in the Mistral-7B Instruct (after BitsandBytes Quantization [Model Compression])

In [6]:
model_name='mistralai/Mistral-7B-Instruct-v0.1'

model_config = transformers.AutoConfig.from_pretrained(
    model_name,
)

tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.47k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/72.0 [00:00<?, ?B/s]

In [7]:
use_4bit = True

# Compute dtype for 4-bit base models
bnb_4bit_compute_dtype = "float16"

# Quantization type (fp4 or nf4)
bnb_4bit_quant_type = "nf4"

# Activate nested quantization for 4-bit base models (double quantization)
use_nested_quant = False

compute_dtype = getattr(torch, bnb_4bit_compute_dtype)

bnb_config = BitsAndBytesConfig(
    load_in_4bit=use_4bit,
    bnb_4bit_quant_type=bnb_4bit_quant_type,
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=use_nested_quant,
)

# Check GPU compatibility with bfloat16
if compute_dtype == torch.float16 and use_4bit:
    major, _ = torch.cuda.get_device_capability()
    if major >= 8:
        print("=" * 80)
        print("Your GPU supports bfloat16: accelerate training with bf16=True")
        print("=" * 80)

In [8]:
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
)


def print_number_of_trainable_model_parameters(model):
    trainable_model_params = 0
    all_model_params = 0
    for _, param in model.named_parameters():
        all_model_params += param.numel()
        if param.requires_grad:
            trainable_model_params += param.numel()
    return f"trainable model parameters: {trainable_model_params}\nall model parameters: {all_model_params}\npercentage of trainable model parameters: {100 * trainable_model_params / all_model_params:.2f}%"

print(print_number_of_trainable_model_parameters(model))

text_generation_pipeline = pipeline(
    model=model,
    tokenizer=tokenizer,
    task="text-generation",
    temperature=0.2,
    repetition_penalty=1.1,
    return_full_text=True,
    max_new_tokens=1000,
)

mistral_llm = HuggingFacePipeline(pipeline=text_generation_pipeline)

`low_cpu_mem_usage` was None, now set to True since model is quantized.


model.safetensors.index.json:   0%|          | 0.00/25.1k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.94G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

trainable model parameters: 262410240
all model parameters: 3752071168
percentage of trainable model parameters: 6.99%


#### Step 3: Incorporate Conversation History and Chaining

In [88]:
import warnings
warnings.filterwarnings("ignore")

standalone_query_generation_pipeline = pipeline(
 model=model,
 tokenizer=tokenizer,
 task="text-generation",
 temperature=1.2,
 repetition_penalty=1.1,
 return_full_text=True,
 max_new_tokens=3000,
)
standalone_query_generation_llm = HuggingFacePipeline(pipeline=standalone_query_generation_pipeline)

response_generation_pipeline = pipeline(
 model=model,
 tokenizer=tokenizer,
 task="text-generation",
 temperature=1.2,
 repetition_penalty=1.1,
 return_full_text=True,
 max_new_tokens=3000,
)
response_generation_llm = HuggingFacePipeline(pipeline=response_generation_pipeline)

In [119]:

_template = """
[INST]
Given the following conversation and a follow up question, rephrase the follow up question to be a standalone question for C++ help.
If you do not see any chat history, you MUST return the "Follow Up Input"
If this is the second question onwards, you should properly rephrase the question.

Here is the chat history and input question.

Chat History:
{chat_history}

Follow Up Input: {question}
[/INST]
Standalone Question:
"""
CONDENSE_QUESTION_PROMPT = PromptTemplate.from_template(_template)

In [104]:
template = """
[INSTRUCT]
Now that you have the standalone question, given the following context and question, answer as a helpful teaching assistant for C++, aiding in programming questions. YOU ARE ALLOWED TO FURTHER QUIZ THE STUDENT ON THEIR UNDERSTANDING IF THEY LACK IN UNDERSTANDING IN YOUR OPINION. :
Question: {question}

Context: {context}

[/INSTRUCT]
ANSWER:
"""
ANSWER_PROMPT = ChatPromptTemplate.from_template(template)

In [105]:
DEFAULT_DOCUMENT_PROMPT = PromptTemplate.from_template(template="{page_content}")

def _combine_documents(
    docs, document_prompt=DEFAULT_DOCUMENT_PROMPT, document_separator="\n\n"
):
    doc_strings = [format_document(doc, document_prompt) for doc in docs]
    return document_separator.join(doc_strings)

In [113]:
# Instantiate ConversationBufferMemory
memory = ConversationBufferMemory(
 return_messages=True, output_key="answer", input_key="question"
)

# First we add a step to load memory
# This adds a "memory" key to the input object
loaded_memory = RunnablePassthrough.assign(
    chat_history=RunnableLambda(memory.load_memory_variables) | itemgetter("history"),
)
# Now we calculate the standalone question
standalone_question = {
    "standalone_question": {
        "question": lambda x: x["question"],
        "chat_history": lambda x: get_buffer_string(x["chat_history"]),
    }
    | CONDENSE_QUESTION_PROMPT
    | standalone_query_generation_llm,
}
# Now we retrieve the documents
retrieved_documents = {
    "docs": itemgetter("standalone_question") | retriever,
    "question": lambda x: x["standalone_question"],
}
# Now we construct the inputs for the final prompt
final_inputs = {
    "context": lambda x: _combine_documents(x["docs"]),
    "question": itemgetter("question"),
}
# And finally, we do the part that returns the answers
answer = {
    "answer": final_inputs | ANSWER_PROMPT,
    "question": itemgetter("question"),
    "context": final_inputs["context"]
}
# And now we put it all together!
final_chain = loaded_memory | standalone_question | retrieved_documents | answer

In [114]:
def call_conversational_rag(question, chain, memory):
    """
    Calls a conversational RAG (Retrieval-Augmented Generation) model to generate an answer to a given question.

    This function sends a question to the RAG model, retrieves the answer, and stores the question-answer pair in memory
    for context in future interactions.

    Parameters:
    question (str): The question to be answered by the RAG model.
    chain (LangChain object): An instance of LangChain which encapsulates the RAG model and its functionality.
    memory (Memory object): An object used for storing the context of the conversation.

    Returns:
    dict: A dictionary containing the generated answer from the RAG model.
    """

    # Prepare the input for the RAG model
    inputs = {"question": question}

    # Invoke the RAG model to get an answer
    result = chain.invoke(inputs)
    result = response_generation_llm.invoke(result["answer"])
    # Save the current question and its answer to memory for future context
    memory.save_context(inputs, {"answer": result.split("ANSWER:")[1]})

    # Return the result
    return result.split("ANSWER:")[1]

In [115]:
question = "What is function overloading?"
print(call_conversational_rag(question, final_chain, memory))


Function overloading is a feature in C++ where multiple functions with the same name but different arguments can be defined. When an object of a class is passed as an argument to a function, the function that matches the type of the argument is called. In other words, the function that has the same number and types of arguments as the object being passed is invoked. This allows for polymorphism and makes the code more flexible and reusable.


In [116]:
question = "How to overload a void function?"
print(call_conversational_rag(question, final_chain, memory))


To overload a void function in C++, you need to define multiple functions with the same name but different parameter types or numbers of parameters. These functions will be called based on the type and number of arguments passed to them. Here is an example:
```c++
#include <iostream>
using namespace std;

// Function overloading with void return type
void printOnNewLine( int x ) {
    cout << "Integer: " << x << endl;
}

void printOnNewLine( char *x ) {
    cout << "String: " << x << endl;
}

int main() {
    int num = 5;
    printOnNewLine( num );
    char str[] = "Hello World!";
    printOnNewLine( str );
    return 0;
}
```
In this example, we have defined two functions with the same name `printOnNewLine`, but with different parameter types. The first function takes an integer parameter and prints "Integer: " followed by the value of the integer. The second function takes a character string parameter and prints "String: " followed by the value of the string.

When we call the `prin

In [117]:
question = "Is overloading for operator similar to this?"
print(call_conversational_rag(question, final_chain, memory))


Yes, overloading operators is similar to function overloading in C++. Just like function overloading, operator overloading involves defining multiple functions with the same name but different parameter types or numbers of parameters. However, instead of using different parameter types, operator overloading uses different operators such as +, -, *, /, etc. The operator overloaded function is called when the corresponding operator is used on an object of the class.


In [118]:
question = "But how is it different?"
print(call_conversational_rag(question, final_chain, memory))



Function overloading and operator overloading are both techniques used in C++ to allow for polymorphism and make code more flexible and reusable. However, there are some key differences between the two.

Function overloading involves defining multiple functions with the same name but different parameter types or numbers of parameters. These functions are called based on the type and number of arguments passed to them. For example, you could define two functions with the same name `printOnNewLine`, one that takes an integer parameter and another that takes a character string parameter. When you call the `printOnNewLine` function, the compiler will automatically determine which function to call based on the type and number of arguments passed to it.

Operator overloading, on the other hand, involves defining multiple functions with the same name but different operators such as +, -, *, /, etc. The operator overloaded function is called when the corresponding operator is used on an obje

The last part of this series (next notebook) will incorporate guardrails (input, output and perhaps also for retrieval) and then deploy it locally using gradio.
Please NOTE that we are not able to deploy this model properly using a dedicated server due to:

1) GPU requirements

2) Model memory requirements


We present an alternative model (in the MERN App folder) which uses Gemma API and builds Guardrails from Scratch + Incorporating the Q/A format promised in the proposal. The base performance of Gemma on the queries was examplary enough to not require RAG. We have previously similarly found (refer to the report) finetuning (especially when tuning data is limited) to be disadvantageous instead of helpful.

This model (as you will see in the next notebook) is relatively underdeveloped as we didn't work on it further after realizing its deployment issues. It is still our initial model and works fine as an initial version with most but not all capabiltieis promised.